<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/BERTGCNAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Setup and Data Loading



In [7]:
# Install dependencies
!pip install torch torch-geometric transformers imbalanced-learn -q

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy import io as sio

# Mount Google Drive
drive.mount('/content/drive')

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Projects/Hayat/facebook-fact-check.csv', encoding='latin-1')

# Extract network features and handle NaN
network_features = df[['share_count', 'reaction_count', 'comment_count']].fillna(0).values  # Replace NaN with 0
print("NaN in network features before filling:", np.isnan(network_features).any())  # Should be True if NaN exists
print("NaN in network features after filling:", np.isnan(network_features).any())  # Should be False

# Standardize network features
scaler = StandardScaler()
X_net_std = scaler.fit_transform(network_features)  # (2282, 3)
print("NaN in X_net_std:", np.isnan(X_net_std).any())  # Should be False

# Save standardized features
sio.savemat('network.mat', {'X_net_std': X_net_std})
print("Network features shape:", X_net_std.shape)

# Prepare labels (binary classification)
labels = df['Rating'].apply(lambda x: 0 if x == 'mostly true' else 1).values  # 0: mostly true, 1: others
y = np.array(labels)
print("Label distribution:", np.bincount(y))  # [1669, 613]

# Move files to Google Drive
!mv /content/network.mat /content/drive/MyDrive/Projects/Hayat/
print("Network features saved to Google Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
NaN in network features before filling: False
NaN in network features after filling: False
NaN in X_net_std: False
Network features shape: (2282, 3)
Label distribution: [1669  613]
Network features saved to Google Drive


# Step 2: Social Branch (Graph Construction and GCN Embeddings)



In [2]:
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops

# Load standardized network features
X_net_std = sio.loadmat('/content/drive/MyDrive/Projects/Hayat/network.mat')['X_net_std']

# Construct graph
G = nx.Graph()
for idx in range(len(df)):
    G.add_node(idx, features=X_net_std[idx])
account_groups = df.groupby('account_id').indices
for account_id, indices in account_groups.items():
    indices = list(indices)
    for i in range(len(indices)):
        for j in range(i + 1, len(indices)):
            G.add_edge(indices[i], indices[j])
print("Graph nodes:", G.number_of_nodes(), "Edges:", G.number_of_edges())

# Prepare GNN data
edges = list(G.edges)
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
x = torch.tensor(X_net_std, dtype=torch.float)
data = Data(x=x, edge_index=edge_index)
edge_index, _ = add_self_loops(data.edge_index, num_nodes=data.num_nodes)
data.edge_index = edge_index
print("GNN Data:", data)

# Define GCN model
class GCN(nn.Module):
    def __init__(self, in_channels=3, hidden_channels=64, out_channels=128):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.relu = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Compute GCN embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gcn_model = GCN().to(device)
data = data.to(device)
gcn_model.eval()
with torch.no_grad():
    gcn_embeddings = gcn_model(data)  # (2282, 128)
print("GCN Embeddings shape:", gcn_embeddings.shape)

# Save GCN model and embeddings
torch.save(gcn_model.state_dict(), 'gcn_model.pth')
torch.save(gcn_embeddings.cpu(), 'gcn_embeddings.pt')
print("GCN model and embeddings saved")

# Move to Google Drive
!mv /content/gcn_model.pth /content/drive/MyDrive/Projects/Hayat/
!mv /content/gcn_embeddings.pt /content/drive/MyDrive/Projects/Hayat/
print("GCN model and embeddings moved to Google Drive")

Graph nodes: 2282 Edges: 368312
GNN Data: Data(x=[2282, 3], edge_index=[2, 370594])
GCN Embeddings shape: torch.Size([2282, 128])
GCN model and embeddings saved
GCN model and embeddings moved to Google Drive


# Step 3: Text Branch (BERT + Attention Embeddings)



In [3]:
from transformers import BertTokenizer, BertModel

# Define Attention Layer
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_dim, 1)

    def forward(self, embeddings):
        # embeddings: (batch_size, seq_len, hidden_dim)
        scores = self.attention(embeddings)  # (batch_size, seq_len, 1)
        scores = torch.softmax(scores, dim=1)  # (batch_size, seq_len, 1)
        context = torch.sum(embeddings * scores, dim=1)  # (batch_size, hidden_dim)
        return context

# BERT Setup
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
bert_model.eval()
attention_layer = Attention(hidden_dim=768).to(device)

# Process texts with BERT + Attention
batch_size = 32
bert_embeddings = []
texts = df['Context Post'].fillna("").tolist()

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=117)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
        token_embeddings = outputs.last_hidden_state  # (batch_size, seq_len, 768)
        context_vector = attention_layer(token_embeddings)  # (batch_size, 768)
        bert_embeddings.append(context_vector.cpu())

# Concatenate all batches
bert_embeddings = torch.cat(bert_embeddings, dim=0)  # (2282, 768)
print("BERT Embeddings with Attention shape:", bert_embeddings.shape)

# Save embeddings
torch.save(bert_embeddings, 'bert_embeddings_with_attention.pt')
!mv /content/bert_embeddings_with_attention.pt /content/drive/MyDrive/Projects/Hayat/
print("BERT embeddings with Attention saved to Google Drive")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT Embeddings with Attention shape: torch.Size([2282, 768])
BERT embeddings with Attention saved to Google Drive


# Step 4: Combine Embeddings and Address Class Imbalance



In [4]:
from imblearn.over_sampling import SMOTE

# Load embeddings
gcn_embeddings = torch.load('/content/drive/MyDrive/Projects/Hayat/gcn_embeddings.pt')  # (2282, 128)
bert_embeddings = torch.load('/content/drive/MyDrive/Projects/Hayat/bert_embeddings_with_attention.pt')  # (2282, 768)

# Combine embeddings
combined_embeddings = torch.cat((gcn_embeddings, bert_embeddings), dim=1)  # (2282, 896)
print("Combined embeddings shape:", combined_embeddings.shape)

# Address class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(combined_embeddings.numpy(), y)
X_resampled = torch.tensor(X_resampled, dtype=torch.float)
y_resampled = torch.tensor(y_resampled, dtype=torch.long)
print("Resampled data shape:", X_resampled.shape, "Label distribution:", np.bincount(y_resampled))

Combined embeddings shape: torch.Size([2282, 896])


ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [6]:
# Check for NaN in embeddings
print("NaN in gcn_embeddings:", torch.isnan(gcn_embeddings).any().item())
print("NaN in bert_embeddings:", torch.isnan(bert_embeddings).any().item())
print("NaN in combined_embeddings:", torch.isnan(combined_embeddings).any().item())

NaN in gcn_embeddings: True
NaN in bert_embeddings: False
NaN in combined_embeddings: True
